In [1]:
import os
import sys
import subprocess
import datetime as dt
import itertools
import numpy as np
import xarray as xr
from dask.distributed import Client, LocalCluster
import dask.array as da
import matplotlib.pyplot as plt
from MHW_daily_calcs import workdir, mdirC5, fnameCanESMClim

In [2]:
workdir='/space/hall5/sitestore/eccc/crd/ccrn/users/reo000/work/MHW_daily/'
mdirC5='/space/hall5/sitestore/eccc/crd/ccrn/users/reo000/data/predictions/cansipsv3_daily/CanESM5'

In [102]:
fnameCanESMAnomByLead=lambda mdir, climyfirst, climylast, ilead, istartlat: \
    f"{mdir}/byLead/anomByLead_cwao_CanESM5.1p1bc-v20240611_hindcast_C{climyfirst:04}_{climylast:04}_L{ilead:03}_j{istartlat:03}_ocean_1d_surface_tso.nc"
fnameCanESMAnomDetrByLeadIndiv=lambda mdir, climyfirst, climylast, ilead, istartlat: f"{mdir}/byLeadDetrIndiv/anomDetrByLead_cwao_CanESM5.1p1bc-v20240611_hindcast_C{climyfirst:04}_{climylast:04}_L{ilead:03}_j{istartlat:03}_ocean_1d_surface_tso.nc"
fnameCanESMAnomDetrByLead=lambda mdir, climyfirst, climylast, ilead, istartlat: f"{mdir}/byLeadDetr/anomDetrByLead_cwao_CanESM5.1p1bc-v20240611_hindcast_"\
    f"C{climyfirst:04}_{climylast:04}_L{ilead:03}_j{istartlat:03}_ocean_1d_surface_tso.nc"
fnameCanESMAnomDetrQtile=lambda mdir, climyfirst, climylast, ilead, istartlat, qt: f"{mdir}/byLeadDetr/qtileDetrByLead_cwao_CanESM5.1p1bc-v20240611_hindcast_C{climyfirst:04}_{climylast:04}_L{ilead:03}_j{istartlat:03}_q{'{:.2f}'.format(qtile).replace('.','_')}_ocean_1d_surface_tso.nc"
fnameCanESMAnomQtile=lambda mdir, climyfirst, climylast, ilead, istartlat, qt: f"{mdir}/byLead/qtileByLead_cwao_CanESM5.1p1bc-v20240611_hindcast_C{climyfirst:04}_{climylast:04}_L{ilead:03}_j{istartlat:03}_q{'{:.2f}'.format(qtile).replace('.','_')}_ocean_1d_surface_tso.nc"
fnameCanESMMHWDetr=lambda mdir, climyfirst, climylast, ilead, istartlat, qt: f"{mdir}/byLeadMHWDetr/MHWDetrByLead_cwao_CanESM5.1p1bc-v20240611_hindcast_C{climyfirst:04}_{climylast:04}_L{ilead:03}_j{istartlat:03}_q{'{:.2f}'.format(qtile).replace('.','_')}_ocean_1d_surface_tso.nc"
fnameCanESMMHW=lambda mdir, climyfirst, climylast, ilead, istartlat, qt: f"{mdir}/byLeadMHW/MHWByLead_cwao_CanESM5.1p1bc-v20240611_hindcast_C{climyfirst:04}_{climylast:04}_L{ilead:03}_j{istartlat:03}_q{'{:.2f}'.format(qtile).replace('.','_')}_ocean_1d_surface_tso.nc"

In [103]:
fnameCanESMAnomDetrByLead(workdir,1993,2023,15,0)

'/space/hall5/sitestore/eccc/crd/ccrn/users/reo000/work/MHW_daily//byLeadDetr/anomDetrByLead_cwao_CanESM5.1p1bc-v20240611_hindcast_C1993_2023_L015_j000_ocean_1d_surface_tso.nc'

In [4]:
climyrs=[1993,2023]
startm=1
qtile=0.9

In [5]:
cm1=plt.get_cmap('jet')
cm2=plt.get_cmap('Grays')
import matplotlib as mpl
normalize = mpl.colors.Normalize(vmin=0, vmax=19)

In [6]:
ilead=5; jj=0
ff=xr.open_dataset(fnameCanESMAnomDetrByLead(workdir, climyrs[0], climyrs[-1], ilead, jj),decode_times=False)
ff

<xarray.Dataset> Size: 1GB
Dimensions:   (lat: 60, lon: 360, reftime: 378, r: 20)
Coordinates:
  * lat       (lat) float64 480B -89.5 -88.5 -87.5 -86.5 ... -32.5 -31.5 -30.5
    leadtime  float64 8B ...
  * lon       (lon) float64 3kB 0.5 1.5 2.5 3.5 4.5 ... 356.5 357.5 358.5 359.5
  * reftime   (reftime) int64 3kB 0 31 59 90 120 ... 11382 11413 11443 11474
  * r         (r) int64 160B 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19
Data variables:
    sst_an    (reftime, r, lat, lon) float64 1GB ...

In [7]:
L=ff.sst_an.shape[0]
# yy=np.array([int(el/12)+starty for el in range(0,L)])
# mm=np.array([int(el%12)+startm for el in range(0,L)])

In [8]:
fc=ff.sst_an.coarsen(reftime=12,boundary='pad').construct(reftime=('year','month'))
fc

<xarray.DataArray 'sst_an' (year: 32, month: 12, r: 20, lat: 60, lon: 360)> Size: 1GB
array([[[[[            nan,             nan,             nan, ...,
                       nan,             nan,             nan],
          [            nan,             nan,             nan, ...,
                       nan,             nan,             nan],
          [            nan,             nan,             nan, ...,
                       nan,             nan,             nan],
          ...,
          [ 1.72244547e+00,  1.88143232e+00,  1.92026047e+00, ...,
            1.23746452e+00,  1.57023946e+00,  1.66334808e+00],
          [ 1.90973215e+00,  2.05239851e+00,  2.04599572e+00, ...,
            1.49485392e+00,  1.80556095e+00,  1.83075637e+00],
          [ 1.89592357e+00,  1.92137813e+00,  1.81056649e+00, ...,
            1.64971302e+00,  1.76092788e+00,  1.76161531e+00]],

         [[            nan,             nan,             nan, ...,
                       nan,             nan,             nan],
          [            nan,             nan,             nan, ...,
                       nan,             nan,             nan],
          [            nan,             nan,             nan, ...,
                       nan,             nan,             nan],
...
          [            nan,             nan,             nan, ...,
                       nan,             nan,             nan],
          [            nan,             nan,             nan, ...,
                       nan,             nan,             nan],
          [            nan,             nan,             nan, ...,
                       nan,             nan,             nan]],

         [[            nan,             nan,             nan, ...,
                       nan,             nan,             nan],
          [            nan,             nan,             nan, ...,
                       nan,             nan,             nan],
          [            nan,             nan,             nan, ...,
                       nan,             nan,             nan],
          ...,
          [            nan,             nan,             nan, ...,
                       nan,             nan,             nan],
          [            nan,             nan,             nan, ...,
                       nan,             nan,             nan],
          [            nan,             nan,             nan, ...,
                       nan,             nan,             nan]]]]])
Coordinates:
  * lat       (lat) float64 480B -89.5 -88.5 -87.5 -86.5 ... -32.5 -31.5 -30.5
    leadtime  float64 8B ...
  * lon       (lon) float64 3kB 0.5 1.5 2.5 3.5 4.5 ... 356.5 357.5 358.5 359.5
    reftime   (year, month) float64 3kB 0.0 31.0 59.0 90.0 ... nan nan nan nan
  * r         (r) int64 160B 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19
Dimensions without coordinates: year, month

In [9]:
fc.reftime.year

<xarray.DataArray 'year' (year: 32)> Size: 256B
array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31])
Coordinates:
    leadtime  float64 8B ...
Dimensions without coordinates: year

In [10]:
fc.reftime.month

<xarray.DataArray 'month' (month: 12)> Size: 96B
array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])
Coordinates:
    leadtime  float64 8B ...
Dimensions without coordinates: month

In [11]:
test=fc.sel(month=[11,0,1])
test

<xarray.DataArray 'sst_an' (year: 32, month: 3, r: 20, lat: 60, lon: 360)> Size: 332MB
array([[[[[            nan,             nan,             nan, ...,
                       nan,             nan,             nan],
          [            nan,             nan,             nan, ...,
                       nan,             nan,             nan],
          [            nan,             nan,             nan, ...,
                       nan,             nan,             nan],
          ...,
          [-7.86105680e-01, -8.12961627e-01, -8.01608379e-01, ...,
           -8.06818310e-01, -7.52711816e-01, -7.52086456e-01],
          [-8.78957437e-01, -9.09146563e-01, -9.27674194e-01, ...,
           -9.15282057e-01, -8.32193549e-01, -8.16033498e-01],
          [-9.37800516e-01, -9.50861073e-01, -9.63739125e-01, ...,
           -9.97070717e-01, -8.89806023e-01, -8.82139743e-01]],

         [[            nan,             nan,             nan, ...,
                       nan,             nan,             nan],
          [            nan,             nan,             nan, ...,
                       nan,             nan,             nan],
          [            nan,             nan,             nan, ...,
                       nan,             nan,             nan],
...
          [ 1.25247477e-01,  1.11907178e-01,  4.81604426e-02, ...,
           -2.97145831e-01, -3.17515373e-01, -1.49960681e-01],
          [ 4.41177202e-01,  4.98276915e-01,  3.98175176e-01, ...,
           -3.39978349e-01, -1.09524325e-01,  1.97339078e-01],
          [ 9.10521076e-01,  8.65675810e-01,  7.04659918e-01, ...,
            1.38375748e-01,  4.24414323e-01,  7.06455629e-01]],

         [[            nan,             nan,             nan, ...,
                       nan,             nan,             nan],
          [            nan,             nan,             nan, ...,
                       nan,             nan,             nan],
          [            nan,             nan,             nan, ...,
                       nan,             nan,             nan],
          ...,
          [ 2.58861064e-01,  2.47893506e-01,  2.21301922e-01, ...,
           -2.58167254e-01, -2.53024101e-01, -4.44308955e-02],
          [ 6.09351946e-01,  6.42716612e-01,  5.82905706e-01, ...,
           -2.91264664e-01, -3.18265713e-02,  3.33912870e-01],
          [ 1.02257399e+00,  1.07885635e+00,  9.16932562e-01, ...,
            1.01167191e-01,  4.75126909e-01,  7.49988954e-01]]]]])
Coordinates:
  * lat       (lat) float64 480B -89.5 -88.5 -87.5 -86.5 ... -32.5 -31.5 -30.5
    leadtime  float64 8B ...
  * lon       (lon) float64 3kB 0.5 1.5 2.5 3.5 4.5 ... 356.5 357.5 358.5 359.5
    reftime   (year, month) float64 768B 334.0 0.0 31.0 ... 1.132e+04 1.135e+04
  * r         (r) int64 160B 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19
Dimensions without coordinates: year, month

In [12]:
test.month

<xarray.DataArray 'month' (month: 3)> Size: 24B
array([0, 1, 2])
Coordinates:
    leadtime  float64 8B ...
Dimensions without coordinates: month

In [13]:
(fc[5,11,5,-1,100]-test[5,0,5,-1,100]).values,(fc[5,0,5,-1,100]-test[5,1,5,-1,100]).values,(fc[5,1,5,-1,100]-test[5,2,5,-1,100]).values

(array(0.), array(0.), array(0.))

In [14]:
sh=fc.shape
def getind(i0):
    if i0>=1 and i0<=10:
        return [i0-1,i0,i0+1]
    elif i0==0:
        return [11,0,1]
    elif i0==11:
        return [10,11,0]
ql=np.nan*np.ones((12,sh[-2],sh[-1]))
for ii in range(0,12):
    pool=fc.sel(month=getind(ii)).values.reshape((sh[0]*3*sh[2],sh[3],sh[4]))
    ql[ii,...]=np.quantile(pool,0.9,axis=0)

In [15]:
ql.shape

(12, 60, 360)

In [16]:
fqout=fnameCanESMAnomDetrQtile(workdir, climyrs[0], climyrs[-1], ilead, jj,qtile)

In [17]:
fqout

'/space/hall5/sitestore/eccc/crd/ccrn/users/reo000/work/MHW_daily//byLeadDetr/qtilemDetrByLead_cwao_CanESM5.1p1bc-v20240611_hindcast_C1993_2023_L005_j000_q0_90_ocean_1d_surface_tso.nc'

In [19]:
dsqt=xr.Dataset(data_vars={'qt':(['month','lat','lon'],ql,{'long_name':f"{100*qtile}th percentile value"}),},
               coords={'month':np.arange(0,12),
                       'lat':ff.lat,
                       'lon':ff.lon})
dsqt.to_netcdf(fqout,mode='w')

In [33]:
qt2=np.expand_dims(ql,[0,2])
qt2.shape

(1, 12, 1, 60, 360)

In [46]:
MHW=np.ma.masked_where(np.isnan(fc),np.where(fc>qt2,1,0))
# v2=np.where(fc>np.tile(qt2,[sh[0],1,sh[2],1,1]),1,0) #gives same result
# np.max(np.abs(v2-v1)) # = 0

In [48]:
daMHW=xr.DataArray(MHW,
                  dims=['year','month','r','lat','lon'],
                   coords={'year':fc.reftime.year.data,'month':fc.reftime.month.data,'r':fc.r.data,
                           'lat':fc.lat.data,'lon':fc.lon.data,'reftime':(['year','month'],fc.reftime.data)})

In [74]:
MHWstack=np.reshape(MHW,(sh[0]*sh[1],sh[2],sh[3],sh[4]))
MHWstack=MHWstack[:ff.sst_an.shape[0],...] # remove appended nans

In [92]:
dsMHW=xr.Dataset(data_vars={'isMHW':(['reftime','r','lat','lon'],MHWstack),
                            'MHW_prob':(['reftime','lat','lon'],np.mean(MHWstack,axis=1))},
                coords={'reftime':ff.reftime,'r':ff.r,'lat':ff.lat,'lon':ff.lon,'leadtime':ff.leadtime})

In [93]:
dsMHW

<xarray.Dataset> Size: 1GB
Dimensions:   (reftime: 378, r: 20, lat: 60, lon: 360)
Coordinates:
    leadtime  float64 8B 135.0
  * reftime   (reftime) int64 3kB 0 31 59 90 120 ... 11382 11413 11443 11474
  * r         (r) int64 160B 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19
  * lat       (lat) float64 480B -89.5 -88.5 -87.5 -86.5 ... -32.5 -31.5 -30.5
  * lon       (lon) float64 3kB 0.5 1.5 2.5 3.5 4.5 ... 356.5 357.5 358.5 359.5
Data variables:
    isMHW     (reftime, r, lat, lon) float64 1GB nan nan nan nan ... 0.0 0.0 0.0
    MHW_prob  (reftime, lat, lon) float64 65MB nan nan nan nan ... 0.0 0.0 0.0

In [100]:
# confirm reshape success
anomfinal=np.reshape(fc.data,(sh[0]*sh[1],sh[2],sh[3],sh[4]))
anomfinal=anomfinal[:ff.sst_an.shape[0],...]

In [98]:
np.max(np.abs(ff.sst_an-anomfinal))

<xarray.DataArray 'sst_an' ()> Size: 8B
np.float64(0.0)
Coordinates:
    leadtime  float64 8B 135.0